In [2]:
# Import Libraries
import plotly.express as p
import warnings
import gc
import random
import numpy as np  
random_state = 123
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


In [3]:
# Memory reduction helper function:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:  # columns
        col_type = df[col].dtypes
        if col_type in numerics:  # numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
            end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
    

In [4]:
# Import dataset


path_to_dataset = r'C:/Alvin'
train = pd.read_csv(path_to_dataset + '/train.csv')
test = pd.read_csv(path_to_dataset + '/test.csv')
extra_data = pd.read_csv(path_to_dataset + '/extra_data.csv')
VariableDefinitions = pd.read_csv(path_to_dataset + '/VariableDefinitions.csv')
submission = pd.read_csv(path_to_dataset + '\SampleSubmission.csv')


## reduce memory usage

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
extra_data = reduce_mem_usage(extra_data)
VariableDefinitions = reduce_mem_usage(VariableDefinitions)


Mem. usage decreased to  0.02 Mb (23.5% reduction)
Mem. usage decreased to  0.03 Mb (25.9% reduction)
Mem. usage decreased to  0.65 Mb (23.6% reduction)
Mem. usage decreased to  0.00 Mb (0.0% reduction)


In [5]:
### Understand the column names and their meanings

for i in range(10):
    print(VariableDefinitions.loc[i, 'COLUMN_NAME'] + ":", VariableDefinitions.loc[i, 'COLUMN_DEFINATION'])



MERCHANT_CATEGORIZED_AT: The time the merchant was categorized by the customer
MERCHANT_NAME: The name of the merchant
MERCHANT_CATEGORIZED_AS: The category the merchant was assigned by the customer
PURCHASE_VALUE: The value of the purchase made by the customer
PURCHASED_AT: The time the purchase was made
IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY: If true indicates that the merchant is not a registered business name
USER_EMAIL: The email of the customer
USER_AGE: The age of the customer
USER_GENDER: The gender of the customer
USER_HOUSEHOLD: The number of family members


In [6]:
#### Get the size of the train and test dataset

print("train shape: ",train.shape, "test shape: ", test.shape, "extra data: ", extra_data.shape)


train shape:  (373, 12) test shape:  (558, 11) extra data:  (10000, 12)


In [7]:
#### Get the number of missing values in the train and test dataset and extra data
print('Missing values for Train:', train.isnull().sum())
print('')
print('Missing values for Test:', test.isnull().sum())
print('')
print('Missing values for Extra data:', extra_data.isnull().sum())


Missing values for Train: MERCHANT_CATEGORIZED_AT                    0
MERCHANT_NAME                              0
MERCHANT_CATEGORIZED_AS                    0
PURCHASE_VALUE                             0
PURCHASED_AT                               0
IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY      0
USER_AGE                                 312
USER_GENDER                                6
USER_HOUSEHOLD                             0
USER_INCOME                                0
USER_ID                                    0
Transaction_ID                             0
dtype: int64

Missing values for Test: MERCHANT_CATEGORIZED_AT                    0
MERCHANT_NAME                              0
PURCHASE_VALUE                             0
PURCHASED_AT                               0
IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY      0
USER_AGE                                 473
USER_GENDER                                5
USER_HOUSEHOLD                             0
USER_INCOME                        

In [8]:
#### To know if the user ids in train dataset are also in test dataset

ids_train = list(train.USER_ID.unique())
ids_test = list(test.USER_ID.unique())

id_train_in_test = [id for id in ids_train if id in ids_test]
id_test_in_train = [id for id in ids_test if id in ids_train]

print("The number of unique user ids that are both in train and test dataset: ",
      len(id_train_in_test))


The number of unique user ids that are both in train and test dataset:  25


In [9]:
#### The unique user ids in test dataset that are not present within train

ids_remaining = [id for id in ids_test if id not in ids_train]
test.loc[test['USER_ID'].isin(ids_remaining)]


,MERCHANT_CATEGORIZED_AT,MERCHANT_NAME,PURCHASE_VALUE,PURCHASED_AT,IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY,USER_AGE,USER_GENDER,USER_HOUSEHOLD,USER_INCOME,USER_ID,Transaction_ID
142,2022-01-07 09:39:55.446642+00,SAFARICOM LIMITED ON,10,2022-01-19 14:57:00+00,False,NaN,Female,9,100000,ID_1AMEATOU,ID_9ozfeujz
185,2022-01-07 09:39:55.446642+00,SAMWEL KIMANI,50,2022-01-20 04:06:00+00,True,NaN,Female,9,100000,ID_1AMEATOU,ID_c7v0jvvs
246,2022-01-07 09:39:55.446642+00,BUY DIRECT LIMITED 2,10386,2021-12-02 14:19:00+00,False,NaN,Female,9,100000,ID_1AMEATOU,ID_gxj2nky1
316,2022-04-12 15:59:14.139347+00,ALLSTAR GAS SUPPLIES,40,2022-04-25 18:01:00+00,False,NaN,Male,8,30000,ID_QIBWKFP0,ID_kws09f8o
400,2021-08-06 11:28:03.289361+00,ALEX SAKWA,100,2021-08-13 09:20:00+00,True,29.0,Male,1,200000,ID_9OCPCLOX,ID_qmxqh91v
460,2022-05-31 13:27:24.704992+00,SAMWEL KIMANI,257,2022-05-29 14:22:00+00,True,32.0,Male,5,80000,ID_KVE1YQI4,ID_u24yokj0
468,2022-05-04 18:39:01.19219+00,DORINE ADHIAMBO,2000,2022-05-11 17:33:00+00,True,NaN,Female,4,36000,ID_90M21S6P,ID_urkqqvp5


* To avoid data leakage you will sort the Dataframe based on USER_ID and PURCHASED AT (which is the time the unique user did a purchase).  When setting your Validation and Test dataset, you will use USER_IDs that the model hasn't seen before


In [10]:
train = train.sort_values(by=['USER_ID', 'PURCHASED_AT'])

In [11]:
#### How balanced is the target variable for the dataset

train.MERCHANT_CATEGORIZED_AS.value_counts()

Bills & Fees        78
Groceries           50
Miscellaneous       43
Data & WiFi         43
Going out           41
Family & Friends    41
Transport & Fuel    29
Shopping            21
Emergency fund      12
Health               6
Loan Repayment       5
Education            2
Rent / Mortgage      2
Name: MERCHANT_CATEGORIZED_AS, dtype: int64

In [12]:
### Since we would be using the extra dataset we need to know the  unique MERCHANT_NAME in the extra dataset that are also in the train dataset

merchants_train = list(train.MERCHANT_NAME.unique())
merchants_extra = list(extra_data.MERCHANT_NAME.unique())
merchants_test = list(test.MERCHANT_NAME.unique())


print("unique size of merchants in train dataset:", len(merchants_train))
print("unique size of merchants in extra dataset:", len(merchants_extra))
print("unique size of merchants in test dataset:", len(merchants_test))


### Merchants name in extra dataset that are also in train dataset 

merchant_extra_in_train = [merchant for merchant in merchants_extra if merchant in merchants_train]
print("merchants names in extra dataset that are also in train dataset: ", len(merchant_extra_in_train))

##### Get the rows of the merchants name that are in the extra dataset and are also in the train dataset

merchant_extra_data_train = extra_data.loc[extra_data['MERCHANT_NAME'].isin(merchant_extra_in_train)]
print("merchant_extra_data shape: ", merchant_extra_data_train.shape)



### Merchants name in extra dataset that are also in test dataset

merchant_extra_in_test = [
    merchant for merchant in merchants_extra if merchant in merchants_test]
print("merchants names in extra dataset that are also in test dataset: ",
      len(merchant_extra_in_test))

# ##### Get the rows of the merchants name that are in the extra dataset and are also in the train dataset

merchant_extra_data_test = extra_data.loc[extra_data['MERCHANT_NAME'].isin(
    merchant_extra_in_test)]
print("merchant_extra_data shape: ", merchant_extra_data_test.shape)


unique size of merchants in train dataset: 218
unique size of merchants in extra dataset: 2781
unique size of merchants in test dataset: 314
merchants names in extra dataset that are also in train dataset:  91
merchant_extra_data shape:  (3243, 12)
merchants names in extra dataset that are also in test dataset:  126
merchant_extra_data shape:  (3656, 12)


In [13]:
###### Get the merchant names in the merchant_extra_in_train list from the train dataset 

merchant_train_data = train.loc[train['MERCHANT_NAME'].isin(
    merchant_extra_in_train)]


##### put the merchant name and the merchant category into two lists and convert them into a single dictionary

list_merchants_name = list(train.MERCHANT_NAME.values) 
list_merchants_category = list(train.MERCHANT_CATEGORIZED_AS.values)
dict_merchants = dict(zip(list_merchants_name, list_merchants_category))

###### Map their values unto the MERCHANT_CATEGORIZED_AS column in the merchant_extra_data

merchant_extra_data_train['MERCHANT_CATEGORIZED_AS'] = merchant_extra_data_train['MERCHANT_NAME'].map(
    dict_merchants)


Cleaning Merchant Names

In [14]:
### Print all unique merchant names in train dataset

print(merchant_extra_data_train.MERCHANT_NAME.unique().tolist())


['POA', 'QUICK MART', 'NAIVAS SUPERMARKET', 'SAFARICOM LIMITED', 'NAIVAS KITENGELA', 'NAIVAS', 'NAIROBI JAVA', 'SAFARICOM POSTPAID', 'PAYTECH LIMITED', 'KAPS PARKING', 'RUBIS', 'PESAPAL', 'NAIROBI JAVA HOUSE SARIT CENTRE', 'JAVA', 'CARREFOUR SRT', 'SAFARICOM POST', 'SAFARICOM POSTPAID BUNDLES', 'FAMILY BANK PESA PAP', 'EQUITY PAYBILL ACCOUNT', 'NABO CAPITAL LTD C2B', 'M-SHWARI ACCOUNT', 'SAFARICOM OFFERS  TUNUKIWA', 'IPAY LTD', 'ASL TD', 'IM BANK C2B', 'EQUITY PAYBILL', 'STATES BARBERSHOP', 'OAKS  CORKS - ONLINE', 'SHELL ST AUSTINS SERVICES STATION', 'KRA - NAIROBI COUNTY REVENUE', 'PAYTECH -THE JUNCTION MALL', 'SHOP AND', 'ZUKU  FOR', 'OILIBYA WAIYAKI', 'DR CECILIA', 'KENGELES-GARDEN BAR', 'ALCHEMIIST - LEGEND BAR', 'THE SPOT POOL BAR', 'SAFARICOM HOME', 'FISH BAY', 'PESAPAL  FOR', 'TUSKYS -T MALL', 'SEVEN EIGHT SIX BUTCHERY', 'THREE BINS SERVICES', 'CAPTON ENTERPRISES NRBI WEST', 'TOTAL K STATE HOUSE 1', 'GALITOS BELLEVUE', 'CAFETERIA AND TIN TIN RESTAURANT', 'FARMERS BUTCHERY', 'MET

In [15]:
### Move user_id as the first column

first_column = train.pop("USER_ID")
train.insert(0, 'USER_ID', first_column)

KAPS PARKING

In [16]:
merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID']== "ID_3JA0MAFB")\
     & (merchant_extra_data_train['MERCHANT_NAME']== "KAPS PARKING"), "MERCHANT_CATEGORIZED_AS"] = "Transport & Fuel"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_ECX9BS4A")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "KAPS PARKING"), "MERCHANT_CATEGORIZED_AS"] = "Going out"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_O8P8YS18")\
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "KAPS PARKING"), "MERCHANT_CATEGORIZED_AS"] = "Bills & Fees"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_Y0386AT9")
     & (merchant_extra_data_train['MERCHANT_NAME']== "KAPS PARKING"), "MERCHANT_CATEGORIZED_AS"] = "Miscellaneous"

SAFARICOM LIMITED

In [17]:
merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_ECX9BS4A")
     & (merchant_extra_data_train['MERCHANT_NAME']== "SAFARICOM LIMITED"), "MERCHANT_CATEGORIZED_AS"] = "Data & WiFi"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_GAQ3PX9G")
     & (merchant_extra_data_train['MERCHANT_NAME']== "SAFARICOM LIMITED"), "MERCHANT_CATEGORIZED_AS"] = "Health"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_GR569FUO")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "SAFARICOM LIMITED"), "MERCHANT_CATEGORIZED_AS"] = "Miscellaneous"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_OZANC6XT")
     & (merchant_extra_data_train['MERCHANT_NAME']== "SAFARICOM LIMITED"), "MERCHANT_CATEGORIZED_AS"] = "Data & WiFi"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_ZX4DCF4K")
     & (merchant_extra_data_train['MERCHANT_NAME']== "SAFARICOM LIMITED"), "MERCHANT_CATEGORIZED_AS"] = "Data & WiFi"

NAIVAS

In [18]:

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_ECX9BS4A")
     & (merchant_extra_data_train['MERCHANT_NAME']== "NAIVAS"), "MERCHANT_CATEGORIZED_AS"] = "Groceries"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_U9WZMGJZ")
     & (merchant_extra_data_train['MERCHANT_NAME']== "NAIVAS"), "MERCHANT_CATEGORIZED_AS"] = "Family & Friends"

CARREFOUR SRT

In [19]:

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_ECX9BS4A")
     & (merchant_extra_data_train['MERCHANT_NAME']== "CARREFOUR SRT"), "MERCHANT_CATEGORIZED_AS"] = "Groceries"


merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_O8P8YS18")
     & (merchant_extra_data_train['MERCHANT_NAME']== "CARREFOUR SRT"), "MERCHANT_CATEGORIZED_AS"] = "Shopping"

EQUITY PAYBILL ACCOUNT

In [20]:

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_L8QMYB09")
     & (merchant_extra_data_train['MERCHANT_NAME']== "EQUITY PAYBILL ACCOUNT"), "MERCHANT_CATEGORIZED_AS"] = "Shopping"


merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_U9WZMGJZ")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "EQUITY PAYBILL ACCOUNT"), "MERCHANT_CATEGORIZED_AS"] = "Bills & Fees"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_Y0386AT9")
     & (merchant_extra_data_train['MERCHANT_NAME']== "EQUITY PAYBILL ACCOUNT"), "MERCHANT_CATEGORIZED_AS"] = "Groceries"

M-SHWARI ACCOUNT

In [21]:

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_1I8XYBWK")
     & (merchant_extra_data_train['MERCHANT_NAME']== "M-SHWARI ACCOUNT"), "MERCHANT_CATEGORIZED_AS"] = "Emergency fund"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_J8O7LHZ2")
     & (merchant_extra_data_train['MERCHANT_NAME']== "M-SHWARI ACCOUNT"), "MERCHANT_CATEGORIZED_AS"] = "Emergency fund"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_KARF7R4R")
     & (merchant_extra_data_train['MERCHANT_NAME']== "M-SHWARI ACCOUNT"), "MERCHANT_CATEGORIZED_AS"] = "Emergency fund"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_UJ0YSYEV")
     & (merchant_extra_data_train['MERCHANT_NAME']== "M-SHWARI ACCOUNT"), "MERCHANT_CATEGORIZED_AS"] = "Emergency fund"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_ZX4DCF4K")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "M-SHWARI ACCOUNT"), "MERCHANT_CATEGORIZED_AS"] = "Bills & Fees"

SAFARICOM OFFERS  TUNUKIWA

In [22]:
merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_GR569FUO")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "SAFARICOM OFFERS  TUNUKIWA"), "MERCHANT_CATEGORIZED_AS"] = "Miscellaneous"


IM BANK C2B

In [23]:
merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_180RJKP4")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "IM BANK C2B"), "MERCHANT_CATEGORIZED_AS"] = "Bills & Fees"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_ECX9BS4A")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "IM BANK C2B"), "MERCHANT_CATEGORIZED_AS"] = "Rent / Mortgage"


TOTAL K STATE HOUSE 1

In [24]:
merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_1I8XYBWK")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "TOTAL K STATE HOUSE 1"), "MERCHANT_CATEGORIZED_AS"] = "Transport & Fuel"


FARMERS BUTCHERY	


In [25]:
merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_O8P8YS18")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "FARMERS BUTCHERY"), "MERCHANT_CATEGORIZED_AS"] = "Groceries"


NHIF

In [26]:
merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_40L9OTIM")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "NHIF"), "MERCHANT_CATEGORIZED_AS"] = "Bills & Fees"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_Y0386AT9")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "NHIF"), "MERCHANT_CATEGORIZED_AS"] = "Health"


SAFARICOM POST PAID

In [27]:
merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_3JA0MAFB")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "SAFARICOM POST PAID"), "MERCHANT_CATEGORIZED_AS"] = "Data & WiFi"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_D8FOVVBB")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "SAFARICOM POST PAID"), "MERCHANT_CATEGORIZED_AS"] = "Bills & Fees"

merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_U9WZMGJZ")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "SAFARICOM POST PAID"), "MERCHANT_CATEGORIZED_AS"] = "Data & WiFi"


NAIVAS LIFESTYLE

In [28]:
merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_GR569FUO")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "NAIVAS LIFESTYLE"), "MERCHANT_CATEGORIZED_AS"] = "Shopping"


merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_ZX4DCF4K")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "NAIVAS LIFESTYLE"), "MERCHANT_CATEGORIZED_AS"] = "Bills & Fees"


KILELESHWA SUPERSHINE

In [29]:
merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_3JA0MAFB")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "KILELESHWA SUPERSHINE"), "MERCHANT_CATEGORIZED_AS"] = "Miscellaneous"


KUNE FOOD LAB KENYA LIMITED	

In [30]:
merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_3JA0MAFB")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "KUNE FOOD LAB KENYA LIMITED"), "MERCHANT_CATEGORIZED_AS"] = "Going out"


merchant_extra_data_train.loc[(merchant_extra_data_train['USER_ID'] == "ID_D8FOVVBB")
                              & (merchant_extra_data_train['MERCHANT_NAME'] == "KUNE FOOD LAB KENYA LIMITED"), "MERCHANT_CATEGORIZED_AS"] = "Miscellaneous"


In [31]:
#### Create a column to signify that the target variable is imputted 

merchant_extra_data_train['generated_target'] = 0
train['generated_target'] = 0

In [32]:
# print(list(extra_data.MERCHANT_NAME.unique()))

In [33]:
## merge the train and the merchant_extra_data together

train_df = pd.concat([train, merchant_extra_data_train], axis=0)
print("the size of train_df :", train_df.shape)

the size of train_df : (3616, 13)


Cleaning the USER DATE column

In [34]:
train_df.isnull().sum()

USER_ID                                     0
MERCHANT_CATEGORIZED_AT                  3243
MERCHANT_NAME                               0
MERCHANT_CATEGORIZED_AS                     0
PURCHASE_VALUE                              0
PURCHASED_AT                                0
IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY       0
USER_AGE                                 3293
USER_GENDER                                13
USER_HOUSEHOLD                              0
USER_INCOME                                 0
Transaction_ID                              0
generated_target                            0
dtype: int64

In [35]:
test.isnull().sum()

MERCHANT_CATEGORIZED_AT                    0
MERCHANT_NAME                              0
PURCHASE_VALUE                             0
PURCHASED_AT                               0
IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY      0
USER_AGE                                 473
USER_GENDER                                5
USER_HOUSEHOLD                             0
USER_INCOME                                0
USER_ID                                    0
Transaction_ID                             0
dtype: int64

In [36]:
#####  Get to look at the number of missing values for ages in merchant_extra_data

print('')
print('Missing values for Extra data:', merchant_extra_data_train.isnull().sum())
print('')

print("we have only {} of true data to predict the age column".format(
train_df.shape[0] - (2981+312)))



Missing values for Extra data: MERCHANT_CATEGORIZED_AT                  3243
MERCHANT_NAME                               0
MERCHANT_CATEGORIZED_AS                     0
PURCHASE_VALUE                              0
PURCHASED_AT                                0
IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY       0
USER_AGE                                 2981
USER_GENDER                                 7
USER_HOUSEHOLD                              0
USER_INCOME                                 0
USER_ID                                     0
Transaction_ID                              0
generated_target                            0
dtype: int64

we have only 323 of true data to predict the age column


In [37]:
### Delete merchant categorized at and remove rows with nan values

train_df_age = train_df.copy() ##### create a copy of train_df

del train_df_age['MERCHANT_CATEGORIZED_AT']
train_df_age.dropna(axis=0, how = 'any', inplace=True)

print("Size of Dataframe to predict age is now: ", train_df_age.shape)

train_df_age.isnull().sum()


Size of Dataframe to predict age is now:  (323, 12)


USER_ID                                  0
MERCHANT_NAME                            0
MERCHANT_CATEGORIZED_AS                  0
PURCHASE_VALUE                           0
PURCHASED_AT                             0
IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY    0
USER_AGE                                 0
USER_GENDER                              0
USER_HOUSEHOLD                           0
USER_INCOME                              0
Transaction_ID                           0
generated_target                         0
dtype: int64

Create the model to predict the age

In [38]:
print(train_df_age.USER_ID.unique())

#### Sort the dataframe by User ID and date

train_df_age.sort_values(by=['USER_ID', 'USER_AGE'], inplace=True)
train_df_age.reset_index(inplace=True)
del train_df_age['index']

train_ids = ["ID_WO19RXAS",
             "ID_FBK8QIYB",
             "ID_CVK8ERW1",
             "ID_KVE1YQI4"]

test_ids = ["ID_U9WZMGJZ", "ID_L8QMYB09"]

##### Use different USER_ID for train and validation set

train_dataset = train_df_age.loc[train_df_age['USER_ID'].isin(train_ids)]
train_dataset.reset_index(inplace=True)
del train_dataset['index']
test_dataset = train_df_age.loc[train_df_age['USER_ID'].isin(test_ids)]
test_dataset.reset_index(inplace=True)
del test_dataset['index']

#### Specify the columns to be used for the training and testing

unuseful_columns = ['MERCHANT_NAME', 'PURCHASED_AT', 'MERCHANT_CATEGORIZED_AS', 'USER_GENDER',
                    'Transaction_ID', 'generated_target', 'USER_AGE', 'USER_ID', 'IS_PURCHASE_PAID_VIA_MPESA_SEND_MONEY',
                    'USER_GENDER',
                    'USER_HOUSEHOLD',
                    'USER_INCOME']
features = [
    feature for feature in train_df_age.columns if feature not in unuseful_columns]
features


#### Change the data type of the features to int and category

# categories = []
# for col in categories:
#     train_dataset[col] = train_dataset[col].astype("category")
#     test_dataset[col] = test_dataset[col].astype("category")
#     train_df[col] = train_df[col].astype("category")


train_dataset['USER_AGE'] = train_dataset['USER_AGE'].astype('int')


['ID_CVK8ERW1' 'ID_L8QMYB09' 'ID_U9WZMGJZ' 'ID_WO19RXAS' 'ID_KVE1YQI4'
 'ID_FBK8QIYB']


In [39]:
features

['PURCHASE_VALUE']

Modelling

In [42]:
# Create the model
from tabnanny import verbose
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

X, y = train_dataset[features], train_dataset['USER_AGE']
kf = KFold(n_splits=4, shuffle=True, random_state=123)
feats = pd.DataFrame({'features': X.columns})
gbm_predictions = []
cv_score_ = 0
oof_preds = np.zeros((train_dataset.shape[0],))
for i, (tr_index, test_index) in enumerate(kf.split(X, y)):
    print()
    print(f'######### FOLD {i+1} / {kf.n_splits} ')
    X_train, y_train = X.iloc[tr_index, :], y[tr_index]
    X_test, y_test = X.iloc[test_index, :], y[test_index]

    # lgb_train = lgb.Dataset(X_train, y_train)
    # lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    # params = {'boosting_type': 'gbdt',
    #           'objective': 'regression',
    #           'max_depth': 8,
    #           'num_leaves': 10,
    #           'learning_rate': 0.5,
    #           'feature_fraction': 0.9
    #           }
    # model = lgb.train(
    #     params,
    #     lgb_train,
    #     num_boost_round=1000,
    #     valid_sets=[lgb_train, lgb_eval],
    #     valid_names=['train', 'valid'],
    #     verbose_eval=0
    # )

    model = lgb.LGBMRegressor(
           learning_rate=0.6,
           max_depth=5,
        #    objective='multiclass',
        #    boosting_type='dart',
        num_leaves=3000,
    )

    model.fit(X_train, y_train,  eval_set=[
        (X_test, y_test)], eval_metric='mae', early_stopping_rounds=200, verbose=200)

    cv_score_ += mean_absolute_error(y_test, model.predict(X_test)) / kf.n_splits
    oof_preds[test_index] = model.predict(X_test)
    # oof_preds[test_index] = model.predict_proba(X_test)

    preds = model.predict(train_df[X_train.columns])
    gbm_predictions.append(preds)
    # feats[f'Fold {i}'] = model.feature_importance()
    feats[f'Fold {i}'] = model.feature_importances_

feats['Importances'] = feats.mean(axis=1)
print(' CV Mean Absolute Error : ', cv_score_)
preds_xgb = np.average(gbm_predictions, axis=0)
print(preds_xgb.shape)




######### FOLD 1 / 4 
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[99]	valid_0's l1: 1.28146	valid_0's l2: 3.45995

######### FOLD 2 / 4 
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[4]	valid_0's l1: 1.21003	valid_0's l2: 3.15118

######### FOLD 3 / 4 
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[4]	valid_0's l1: 1.7957	valid_0's l2: 6.60986

######### FOLD 4 / 4 
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l1: 0.710979	valid_0's l2: 1.47227
 CV Mean Absolute Error :  1.2495428492044955
(3616,)


In [298]:
# 1.249 = num_leaves = 2000, depth = 16, learning_rate = 0.6


In [43]:

train_df['predicted_age'] = preds_xgb
test['predicted_age'] = preds_xgb

# print(train_df.predicted_age.unique())

#### Seeing how accurate the model is
# train_df[train_df['USER_AGE']==32]

In [44]:
####  Specify the rows with imputted age 

generated_list_age =[]
for age in train_df['USER_AGE']:
    if age >=0:
        generated_list_age.append(0)
    else:
        generated_list_age.append(1)

train_df['generated_age']= generated_list_age


generated_list_age =[]
for age in test['USER_AGE']:
    if age >=0:
        generated_list_age.append(0)
    else:
        generated_list_age.append(1)

test['generated_age']= generated_list_age

In [45]:
feats

,features,Fold 0,Fold 1,Fold 2,Fold 3,Importances
0,PURCHASE_VALUE,99,4,4,100,51.75


In [79]:
print("train_df predicted ages value counts :", train_df.predicted_age.value_counts())

print("extra data user age value counts :", extra_data.USER_AGE.value_counts())

print("train dataset user age value counts :", train.USER_AGE.value_counts())



train_df predicted ages value counts : 26.326504    1775
27.925188    1139
26.092356     344
27.294236     304
26.399787      54
Name: predicted_age, dtype: int64
extra data user age value counts : 25.0    404
27.0    341
26.0    172
31.0    136
32.0    104
23.0      1
Name: USER_AGE, dtype: int64
train dataset user age value counts : 25.0    38
27.0    14
26.0     7
32.0     2
Name: USER_AGE, dtype: int64


In [80]:
# save the model to disk

import pickle

filename = 'age_prediction_model_1.249.sav'
pickle.dump(model, open(filename, 'wb'))


# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)


In [46]:
train_df.to_csv(r"C:\Alvin/train_df.csv")

In [46]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 30))
train_df_age.groupby(['USER_AGE', 'PURCHASE_VALUE'])[
    'PURCHASE_VALUE'].count().sort_values(ascending=False).head(40)


USER_AGE  PURCHASE_VALUE
26.0      150               17
27.0      50                11
          200               10
26.0      200               10
          100                9
27.0      99                 9
25.0      50                 9
27.0      1050               8
25.0      20                 7
27.0      100                6
25.0      1000               6
27.0      40                 5
26.0      500                5
27.0      1000               5
25.0      500                5
          100                5
          200                4
27.0      2000               4
          20                 4
25.0      2000               3
          490                3
27.0      500                3
25.0      700                3
27.0      5                  3
31.0      2900               3
27.0      230                2
25.0      1790               2
27.0      130                2
          48                 2
          10                 2
26.0      250                2
27.0      1200

<Figure size 1440x2160 with 0 Axes>

In [47]:
test.to_csv(r"C:\Alvin/test_augment.csv")
